In [2]:
import numpy as np
import netCDF4 as cdf
import pandas as pd
import xesmf as xe
import xarray as xr
import warnings

In [3]:
domains        = 1                                                                    # check this always
w              = '../../../DATA/WRF/wrfinput/'   # check this always
w_geo          = '../../../DATA/WRF/domains/4_domains/'# check this always
pre            = 'wrfinput_d0'
pre2           = 'geo_em.d0'
sim_time       = '2022-08-01 00:00:00','2022-08-31 23:00:00' 
dates          = pd.to_datetime(sim_time[0]).strftime('%Y-%m-%d')
pos            = '.nc'  
projection_wrf = 'Lambert Conformal'
# Revisar el script e inclurile fechas pa saber que 

In [4]:
dom              = domains
wrf_inp          = w + pre  + str(domains)
wrf_geo_p        = w_geo + pre2 + str(domains) + pos

In [5]:
print('STARTING WITH GHG fire emissions REGRID-PREPROCESSING')
wrf_i          = cdf.Dataset(wrf_inp)
wrf_geo        = cdf.Dataset(wrf_geo_p)
wrf_lat        = np.array(wrf_geo.variables['XLAT_M'])
wrf_lon        = np.array(wrf_geo.variables['XLONG_M'])
wrf_lmask      = np.array(wrf_geo.variables['LANDMASK'])
wrf_lat_c      = np.array(wrf_geo.variables['XLAT_C'])
wrf_lon_c      = np.array(wrf_geo.variables['XLONG_C'])

STARTING WITH GHG fire emissions REGRID-PREPROCESSING


In [57]:
wchts_path      = '../../../DATA/sources/ANTHRO/global/wetchart/'
lat             = 'lat'
lon             = 'lon'
regrid_method   = 'conservative'
month           = int(pd.to_datetime(sim_time[0]).strftime('%m'))

In [54]:
ss = cdf.Dataset('../../../DATA/sources/ANTHRO/global/wetchart/wchts_v1-3-1_model_2913_global_wet_ch4_monthly-2009.nc')
data = np.array(ss.variables['emission'])[7]
data[data<=0] = 0
data[np.isnan(data)] = 0
sss = data
sss.max(),sss.min()

(8.900421660558969, 0.0)

In [58]:
import glob

wchts_data = {}
num_model = ['2913','2923','2933','2914','2924','2934']
for nm in num_model:
    files = sorted(glob.glob(wchts_path+'wchts_v1-3-1_model_'+str(nm)+'_global*.nc'))
    means = []
    for f in files:
        wchts_open = cdf.Dataset(f)
        data = np.array(wchts_open.variables['emission'][month-1])
        data[data<=0] = 0
        data[np.isnan(data)] = 0
        means.append(data)
    wchts_data[nm] = np.mean(means, axis=0)
    wchts_data[nm] = wchts_data[nm]*36

lons_wchts   = np.array(wchts_open.variables[lon])
lats_wchts   = np.array(wchts_open.variables[lat])
        

In [59]:
lons_wchts

array([-179.75, -179.25, -178.75, -178.25, -177.75, -177.25, -176.75,
       -176.25, -175.75, -175.25, -174.75, -174.25, -173.75, -173.25,
       -172.75, -172.25, -171.75, -171.25, -170.75, -170.25, -169.75,
       -169.25, -168.75, -168.25, -167.75, -167.25, -166.75, -166.25,
       -165.75, -165.25, -164.75, -164.25, -163.75, -163.25, -162.75,
       -162.25, -161.75, -161.25, -160.75, -160.25, -159.75, -159.25,
       -158.75, -158.25, -157.75, -157.25, -156.75, -156.25, -155.75,
       -155.25, -154.75, -154.25, -153.75, -153.25, -152.75, -152.25,
       -151.75, -151.25, -150.75, -150.25, -149.75, -149.25, -148.75,
       -148.25, -147.75, -147.25, -146.75, -146.25, -145.75, -145.25,
       -144.75, -144.25, -143.75, -143.25, -142.75, -142.25, -141.75,
       -141.25, -140.75, -140.25, -139.75, -139.25, -138.75, -138.25,
       -137.75, -137.25, -136.75, -136.25, -135.75, -135.25, -134.75,
       -134.25, -133.75, -133.25, -132.75, -132.25, -131.75, -131.25,
       -130.75, -130